In [ ]:
# Run this cell to install pip requirements.
!pip install -r requirements.txt

# Part 1: Basic Structures

Overview of steps:

1. Imports
1. Create Some Tensors
1. Basic Tensor Operations
1. Automatic Differentiation
1. Moving to GPU

In [10]:
# Import PyTorch and check version.

import torch
print(torch.__version__)

2.4.0+cu121


In [11]:
# Create a tensor

# Create a 1D tensor (vector)
tensor = torch.tensor([1, 2, 3, 4, 5])
print(tensor)

# Create a 2D tensor (matrix)
tensor_2d = torch.tensor([[1, 2], [3, 4]])
print(tensor_2d)

tensor([1, 2, 3, 4, 5])
tensor([[1, 2],
        [3, 4]])


In [12]:
# Basic Operations

# Element-wise addition
tensor_add = tensor + 2
print(tensor_add)

# Element-wise multiplication
tensor_mul = tensor * 2
print(tensor_mul)

# Matrix multiplication
matrix_mul = torch.matmul(tensor_2d, tensor_2d)
print(matrix_mul)

tensor([3, 4, 5, 6, 7])
tensor([ 2,  4,  6,  8, 10])
tensor([[ 7, 10],
        [15, 22]])


In [13]:
# Automatic Differentiation

# Create a tensor with requires_grad=True to track operations on it.
x = torch.tensor(2.0, requires_grad=True)
y = x**2 + 3*x + 1

# Compute gradients.
y.backward()

# Print the gradient (dy/dx)
print(x.grad)

tensor(7.)


In [14]:
# Moving tensors to GPU.

if torch.cuda.is_available():
    tensor_gpu = tensor.to('cuda')
    print(tensor_gpu)
else:
    print("GPU not available.")

GPU not available.


# Part 2: Training a Model and Doing Inference

Overview of steps:

1. Imports.
1. Create example dataset.
1. Define a linear model: $y=Wx+b$.
   - $W$ is the weight.
   - $b$ is the bias.
1. Train the model using training data. The model adjusts $W$ and $b$ to minimize the loss function.
1. Perform inference with the trained model. Given an arbitrary input, use the model to predict the outputs.

## 1. Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

## 2. Create Example Dataset

In [2]:
# Generate some example data.
# Data will match the following function outputs: y = 2 * x + 1
X = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y = torch.tensor([[3.0], [5.0], [7.0], [9.0]])

## 3. Define a Linear Model 

In [4]:
# Define a simple linear model.
model = nn.Linear(in_features=1, out_features=1)

In [5]:
# Define loss function, optimizer
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

## 4. Train the Model

In [8]:
# Training loop:
epochs = 1000
for epoch in range(epochs):
    # Forward pass: compute predicted y by passing X to the model.
    y_pred = model(X)
    
    # Compute the loss
    loss = loss_fn(y_pred, y)

    # Zero gradients, perofrm a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 100/1000, Loss: 5.8437264669919387e-05
Epoch 200/1000, Loss: 3.2082934922073036e-05
Epoch 300/1000, Loss: 1.7612424926483072e-05
Epoch 400/1000, Loss: 9.66916650213534e-06
Epoch 500/1000, Loss: 5.308143045112956e-06
Epoch 600/1000, Loss: 2.9151124181225896e-06
Epoch 700/1000, Loss: 1.6004523786250502e-06
Epoch 800/1000, Loss: 8.785764293861575e-07
Epoch 900/1000, Loss: 4.828955866287288e-07
Epoch 1000/1000, Loss: 2.6537878738963627e-07


## 4. Perform Inference

In [9]:
# Perform inference.

with torch.no_grad():  # Disable gradient calcualtion for inference.
    new_X = torch.tensor([[5.0]])  # Example input
    predicted_y = model(new_X)
    print(f"Predicted y for input 5.0: {predicted_y.item()}")

Predicted y for input 5.0: 10.99911880493164
